In [1]:
# importing the libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np
from urllib.request import urlopen
import pandas
import sqlite3

import random

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)

connection = sqlite3.connect('D:/Akademik/Semester 7/TA 1/Example 1/airport-develop/airport-develop/resources/database/Airports.db')
query = 'select * from runways'
database = pd.read_sql_query(query, connection)

icao_code_list = database["airportICAO"].unique()
icao_code_list

array(['AGGH', 'ANYN', 'AYBK', ..., 'OTHH', 'OMDW', 'OTBD'], dtype=object)

In [2]:
old_dtypes = database.dtypes
database.iloc[:, 2:] = None
database = database.astype(old_dtypes)

rwy = lambda icao, ident: database.index[(database.airportICAO == icao) & (database.rwyIdent == ident)]


In [3]:
database

,airportICAO,rwyIdent,rwyBearing,rwyTrueBearing,rwySlope,rwyThresElevation,rwyThresCrossHeight,rwyWidth,rwyLon,rwyLat,rwyLength,rwySurface,rwyGlideslopeAngle,ilsLat,ilsLon,ilsElevation,ilsFreq,ilsBearing,ilsIdent,ilsCategory,gldslpLat,gldslpLon,gldslpElevation,gldslpFreq,gldslpAngle,gldslpIdent,llzLat,llzLon,llzElevation,llzFreq,llzBearing,llzIdent,llzCategory
0,AGGH,06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
1,AGGH,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
2,ANYN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
3,ANYN,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
4,AYBK,04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10837,OMDW,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
10838,OTBD,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
10839,OTBD,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
10840,UNNT,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None


In [4]:
# Function - Obtaining Primary Data: Latitude, Longitude, Elevation, Alignment, Slope
def get_primary_data(soup):
    
    #Initialisasi soup untuk data primer dari data mentah
    runway_tables = soup.find_all("table", attrs={"class": "table table-striped table-condensed"})

    #Deklarasi data yang dibutuhkan
    primary_data = {}
    runway_key = {}

    #iterasi tabel data soup aktual (table-condensed)
    for runway_table in runway_tables:
        #ekstrak runway identifier
        runway_head = runway_table.find_all("th")
        #ekstrak data dari setiap runway
        runway_table_data = runway_table.tbody.find_all("tr")
        #iterasi semua tabel runway
        for index, runway_name in enumerate(runway_head):
            #ekstrak nama runway dan pemberian id buatan (kiri:1 kanan:2)
            if "Runway" in runway_name.text:
                primary_data[runway_name.text] = {'position_iteration': index}
                runway_key[runway_name.text] = index
                #ekstrak elemen yang diperlukan dari data primer 
                for r, tr in enumerate(runway_table_data):
                    if r > 4:
                        break 
                    current_data = tr.find_all("td")
                    for i, data in enumerate(current_data):
                        if i == index:
                            primary_data[runway_name.text][current_data[0].text] = current_data[i].text
    
    #Return feedback dataa
    return primary_data

In [5]:
# Function - Obtaining Secondary Data: Length, Width, Surface

def get_secondary_data(soup):
    
    #Initialisasi soup untuk data sekunder dari data mentah
    runway_info = soup.find_all("table", attrs={"class": "table table-striped"})

    #Deklarasi data yang dibutuhkan
    temp_dict = {}
    secondary_data = {}
    dict_index = 0

    #Membuat list auxilliary variabel untuk ekstrak nama runway
    heading_tags = ["h3"]
    for index, tags in enumerate(soup.find_all(heading_tags)):
        if "Runway" in tags.text:
            temp_dict[index] = tags.text.strip()

    #iterasi semua tabel sekunder
    for current_info in runway_info:
        #iterasi elemen dari dalam dari setiap tabel
        for index, tr in enumerate(current_info.find_all("tr")):
            try:
                current_data = tr.find_all("td")
                #ekstrak data dimensi
                if (current_data[0].text == "Dimension:") or (current_data[0].text == "Surface:"):
                    if temp_dict[dict_index] in secondary_data.keys():
                        secondary_data[temp_dict[dict_index]][current_data[0].text] = current_data[1].text
                        dict_index = dict_index + 1
                    else:
                        secondary_data[temp_dict[dict_index]] = {current_data[0].text: current_data[1].text}

            except:
                pass
    
    #Return feedback data
    return secondary_data

In [6]:
# Function - Combining Primary & Secondary Data

def combine_all_data(primary_data, secondary_data):
    #iterasi semua data sekunder 
    for runway_name_string, data in secondary_data.items():
        #ekstrak nama runway aktual 
        runway = runway_name_string.split(" ")[0]              #Runway
        #ekstrak per arah nama runway
        lanes = runway_name_string.split(" ")[1]               #17/8
        #iterasi semua arah nama runway
        for lane in lanes.split("/"):
            runway_name = "{} {}".format(runway, lane)         #runway 17 & runway 8
            #iterasi semua data sekunder 
            for attribute, value in data.items():
                #mengalokasi elemen sekunder ke data primer
                primary_data[runway_name][attribute] = value

    #Return feedback data
    return primary_data

In [7]:
for icao_code in icao_code_list:
    #Step 1: ngambil url & fetch data
    url = "https://www.airport-data.com/world-airports/"+icao_code+"/"
    print("Process for {}".format(url))
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "lxml")

    #step 3: ngambil semua data 
    # Go through all matching tables to extract data
    runway_table = soup.find_all("table", attrs={"class": "table table-striped table-condensed"})
    primary_data = get_primary_data(soup)
    secondary_data = get_secondary_data(soup)
    combined_data = combine_all_data(primary_data, secondary_data)

    #print(combined_data)

    #   #step 4: bersihin data
    #iterasi semua key dan value dari combined data
    clean_data = {}
    for runway_name, value in combined_data.items():
        clean_data[runway_name] = {}
        #iterasi elemen dalam value
        for attribute_name, element in value.items():
            if attribute_name == "Longitude:":
                if "/ E" in element:
                    clean_data[runway_name][attribute_name] = element.split("/ E")[0]
                elif "/ W" in element:
                    clean_data[runway_name][attribute_name] = element.split("/ W")[0]
                else:
                    clean_data[runway_name][attribute_name] = element.split("/ W")[0]
            elif attribute_name == "Latitude:":
                if "/ N" in element:
                    clean_data[runway_name][attribute_name] = element.split("/ N")[0]
                elif "/ S" in element:
                    clean_data[runway_name][attribute_name] = element.split("/ S")[0]
                else:
                    clean_data[runway_name][attribute_name] = element.split("/ S")[0]
            elif attribute_name == "End Elevation:":
                try:
                    clean_data[runway_name][attribute_name] = str(float(element.split(" ft")[0])*0.3048)
                except:
                    pass
                
            elif attribute_name == "Dimension:":
                clean_data[runway_name]["Length"] = element.split(" / ")[1].split(" x ")[0]
                clean_data[runway_name]["Width"] = element.split(" / ")[1].split(" x ")[1].rstrip(element[-1])
                #'Dimension:': '13123 x 197 ft / 3999.9 x 60.0 m',
            elif attribute_name == "position_iteration":
                pass
            else:
                clean_data[runway_name][attribute_name] = element

    #step 5: ngerangkai data ke dataframe

    for (runway, value) in clean_data.items():
        
        #Edit and store runway-ident
        if ('Runway' in runway):
            runwayIdent = runway.split(' ')[1]
        else:
            runwayIdent = runway

        index = rwy(icao_code, runwayIdent)
            
        try:
            database.loc[index, 'rwyBearing'] = value['Alignment:']
        except:
            pass

        try: 
             database.loc[index, 'rwySlope'] = value['Slope:']
        except:
            pass

        try:
             database.loc[index, 'rwyThresElevation'] = value['End Elevation:']
        except:
            pass

        try:
             database.loc[index, 'rwyWidth'] = value['Width']
        except:
            pass

        try:
             database.loc[index, 'rwyLon'] = value['Longitude:']
        except:
            pass

        try:
             database.loc[index, 'rwyLat'] = value['Latitude:']
        except:
            pass

        try:
             database.loc[index, 'rwyLength'] = value['Length']
        except:
            pass

        try:
             database.loc[index, 'rwySurface'] = value['Surface:']
        except:
            pass


Process for https://www.airport-data.com/world-airports/AGGH/
Process for https://www.airport-data.com/world-airports/ANYN/
Process for https://www.airport-data.com/world-airports/AYBK/
Process for https://www.airport-data.com/world-airports/AYDU/
Process for https://www.airport-data.com/world-airports/AYGN/
Process for https://www.airport-data.com/world-airports/AYKI/
Process for https://www.airport-data.com/world-airports/AYKV/
Process for https://www.airport-data.com/world-airports/AYMD/
Process for https://www.airport-data.com/world-airports/AYMO/
Process for https://www.airport-data.com/world-airports/AYMR/
Process for https://www.airport-data.com/world-airports/AYMS/
Process for https://www.airport-data.com/world-airports/AYNZ/
Process for https://www.airport-data.com/world-airports/AYPY/
Process for https://www.airport-data.com/world-airports/AYTK/
Process for https://www.airport-data.com/world-airports/AYWK/
Process for https://www.airport-data.com/world-airports/BGBW/
Process 

Process for https://www.airport-data.com/world-airports/CYHO/
Process for https://www.airport-data.com/world-airports/CYHR/
Process for https://www.airport-data.com/world-airports/CYHU/
Process for https://www.airport-data.com/world-airports/CYHY/
Process for https://www.airport-data.com/world-airports/CYHZ/
Process for https://www.airport-data.com/world-airports/CYIB/
Process for https://www.airport-data.com/world-airports/CYIF/
Process for https://www.airport-data.com/world-airports/CYIK/
Process for https://www.airport-data.com/world-airports/CYIO/
Process for https://www.airport-data.com/world-airports/CYIV/
Process for https://www.airport-data.com/world-airports/CYJN/
Process for https://www.airport-data.com/world-airports/CYJT/
Process for https://www.airport-data.com/world-airports/CYKA/
Process for https://www.airport-data.com/world-airports/CYKD/
Process for https://www.airport-data.com/world-airports/CYKF/
Process for https://www.airport-data.com/world-airports/CYKG/
Process 

Process for https://www.airport-data.com/world-airports/CYVV/
Process for https://www.airport-data.com/world-airports/CYVZ/
Process for https://www.airport-data.com/world-airports/CYWG/
Process for https://www.airport-data.com/world-airports/CYWJ/
Process for https://www.airport-data.com/world-airports/CYWK/
Process for https://www.airport-data.com/world-airports/CYWL/
Process for https://www.airport-data.com/world-airports/CYWP/
Process for https://www.airport-data.com/world-airports/CYWY/
Process for https://www.airport-data.com/world-airports/CYXC/
Process for https://www.airport-data.com/world-airports/CYXD/
Process for https://www.airport-data.com/world-airports/CYXE/
Process for https://www.airport-data.com/world-airports/CYXH/
Process for https://www.airport-data.com/world-airports/CYXJ/
Process for https://www.airport-data.com/world-airports/CYXK/
Process for https://www.airport-data.com/world-airports/CYXL/
Process for https://www.airport-data.com/world-airports/CYXN/
Process 

Process for https://www.airport-data.com/world-airports/DTTA/
Process for https://www.airport-data.com/world-airports/DTTF/
Process for https://www.airport-data.com/world-airports/DTTG/
Process for https://www.airport-data.com/world-airports/DTTJ/
Process for https://www.airport-data.com/world-airports/DTTR/
Process for https://www.airport-data.com/world-airports/DTTX/
Process for https://www.airport-data.com/world-airports/DTTZ/
Process for https://www.airport-data.com/world-airports/DXNG/
Process for https://www.airport-data.com/world-airports/DXXX/
Process for https://www.airport-data.com/world-airports/EBAW/
Process for https://www.airport-data.com/world-airports/EBBR/
Process for https://www.airport-data.com/world-airports/EBCI/
Process for https://www.airport-data.com/world-airports/EBKT/
Process for https://www.airport-data.com/world-airports/EBLG/
Process for https://www.airport-data.com/world-airports/EBOS/
Process for https://www.airport-data.com/world-airports/EDAC/
Process 

Process for https://www.airport-data.com/world-airports/EGKB/
Process for https://www.airport-data.com/world-airports/EGKK/
Process for https://www.airport-data.com/world-airports/EGLC/
Process for https://www.airport-data.com/world-airports/EGLF/
Process for https://www.airport-data.com/world-airports/EGLK/
Process for https://www.airport-data.com/world-airports/EGLL/
Process for https://www.airport-data.com/world-airports/EGMC/
Process for https://www.airport-data.com/world-airports/EGMD/
Process for https://www.airport-data.com/world-airports/EGMH/
Process for https://www.airport-data.com/world-airports/EGNC/
Process for https://www.airport-data.com/world-airports/EGNH/
Process for https://www.airport-data.com/world-airports/EGNJ/
Process for https://www.airport-data.com/world-airports/EGNL/
Process for https://www.airport-data.com/world-airports/EGNM/
Process for https://www.airport-data.com/world-airports/EGNR/
Process for https://www.airport-data.com/world-airports/EGNS/
Process 

Process for https://www.airport-data.com/world-airports/ENSG/
Process for https://www.airport-data.com/world-airports/ENSH/
Process for https://www.airport-data.com/world-airports/ENSK/
Process for https://www.airport-data.com/world-airports/ENSN/
Process for https://www.airport-data.com/world-airports/ENSO/
Process for https://www.airport-data.com/world-airports/ENSR/
Process for https://www.airport-data.com/world-airports/ENSS/
Process for https://www.airport-data.com/world-airports/ENST/
Process for https://www.airport-data.com/world-airports/ENTC/
Process for https://www.airport-data.com/world-airports/ENTO/
Process for https://www.airport-data.com/world-airports/ENVA/
Process for https://www.airport-data.com/world-airports/ENVD/
Process for https://www.airport-data.com/world-airports/ENZV/
Process for https://www.airport-data.com/world-airports/EPBY/
Process for https://www.airport-data.com/world-airports/EPGD/
Process for https://www.airport-data.com/world-airports/EPKK/
Process 

KeyboardInterrupt: 

In [8]:
database.head(5)

,airportICAO,rwyIdent,rwyBearing,rwyTrueBearing,rwySlope,rwyThresElevation,rwyThresCrossHeight,rwyWidth,rwyLon,rwyLat,rwyLength,rwySurface,rwyGlideslopeAngle,ilsLat,ilsLon,ilsElevation,ilsFreq,ilsBearing,ilsIdent,ilsCategory,gldslpLat,gldslpLon,gldslpElevation,gldslpFreq,gldslpAngle,gldslpIdent,llzLat,llzLon,llzElevation,llzFreq,llzBearing,llzIdent,llzCategory
0,AGGH,06,059.0,NaN,-0.2,9.144,NaN,45.1,160.045472,-9.431736,2200.0,"Asphalt, asphaltic concrete, tar macadam, or b...",None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
1,AGGH,24,239.0,NaN,0.2,4.2672,NaN,45.1,160.064106,-9.424264,2200.0,"Asphalt, asphaltic concrete, tar macadam, or b...",None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
2,ANYN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
3,ANYN,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
4,AYBK,04,040.0,NaN,0.0,3.3528000000000002,NaN,29.9,154.667469,-5.427006,1562.1,"Asphalt, asphaltic concrete, tar macadam, or b...",None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None


In [19]:
database.to_parquet('airport-data.com.parquet', compression='gzip')

In [2]:
pd.read_parquet('airport-data.com.parquet').head()

,airportICAO,rwyIdent,rwyBearing,rwyTrueBearing,rwySlope,rwyThresElevation,rwyThresCrossHeight,rwyWidth,rwyLon,rwyLat,rwyLength,rwySurface,rwyGlideslopeAngle,ilsLat,ilsLon,ilsElevation,ilsFreq,ilsBearing,ilsIdent,ilsCategory,gldslpLat,gldslpLon,gldslpElevation,gldslpFreq,gldslpAngle,gldslpIdent,llzLat,llzLon,llzElevation,llzFreq,llzBearing,llzIdent,llzCategory
0,AGGH,06,059.0,None,-0.2,9.144,None,45.1,160.045472,-9.431736,2200.0,"Asphalt, asphaltic concrete, tar macadam, or b...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,AGGH,24,239.0,None,0.2,4.2672,None,45.1,160.064106,-9.424264,2200.0,"Asphalt, asphaltic concrete, tar macadam, or b...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ANYN,12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ANYN,30,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,AYBK,04,040.0,None,0.0,3.3528000000000002,None,29.9,154.667469,-5.427006,1562.1,"Asphalt, asphaltic concrete, tar macadam, or b...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [3]:
ADrequestayah = pd.read_parquet('airport-data.com.parquet').head(100)

ADrequestayah.to_csv('ADayah.csv')

In [8]:
from fastparquet import ParquetFile

# Reading the data from Parquet File
pf = ParquetFile("airport-data.com.parquet")

# Converting data in to pandas dataFrame
dataFrame = pf.to_pandas()

# Converting to CSV
dataFrame.to_csv("airport-data.com.csv", index = False)